In [6]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [2]:
DAYSBACK = 365
VMIN_CORNER = '_H2_0P50V_BLOCK'
VNOM_CORNER = '_H2_0P65V_BLOCK'
VMAX_CORNER = '_H2_1P05V_BLOCK'
QUERY = f"""
WITH cte AS(
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label, 
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND RIGHT(tp.parm_Label,7) NOT IN ('Block26','Block27','Block28')
        AND family_Code = 'Q5'
    ORDER BY lot_Id, wafer_Id, parm_Label)
    
    UNION ALL
    
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X) || '_' || unitcell_Y NOT IN ('1_7','2_9','3_10','4_11','5_11','6_12','7_12','9_11','10_11','11_10','12_8','12_5','11_3','10_2','9_2','6_1','5_2','4_2','3_3','2_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND RIGHT(tp.parm_Label,7) NOT IN ('Block26','Block27','Block28')
        AND family_Code = 'Q6'
    ORDER BY lot_Id, wafer_Id, parm_Label)
),
cte2 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, retx, rety, MIN(yield) AS yield
    FROM cte
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner, retx, rety
),
cte3 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, AVG(yield) AS yield
    FROM cte2
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner
),
cte4 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, MAX(date) AS date, 
    MAX(CASE WHEN corner = '0p50V' THEN yield ELSE -100 END) AS vmin_yield,
    MAX(CASE WHEN corner = '0p65V' THEN yield ELSE -100 END) AS vnom_yield
    FROM cte3
    GROUP BY lot_id_base, lot_id, wafer_id, family_code
)

SELECT lot_id_base, lot_id, wafer_id, family_code, Max(date) AS date, vmin_yield,
    AVG(vnom_yield - vmin_yield) AS vmin_loss
FROM cte4
GROUP BY lot_id_base, lot_id, wafer_id, family_code, vmin_yield
"""
df = ibmdata.isdw.query(QUERY)
df['vmin_yield'] = df['vmin_yield'].astype(float).round(2)
df['vmin_loss'] = df['vmin_loss'].astype(float).round(2)

In [3]:
df.to_csv('~/Downloads/Sail_Vmin_no825TESLATS.csv')

W/O Blocks 1-5 as well

In [4]:
DAYSBACK = 365
VMIN_CORNER = '_H2_0P50V_BLOCK'
VNOM_CORNER = '_H2_0P65V_BLOCK'
VMAX_CORNER = '_H2_1P05V_BLOCK'
EXCLUDE = """'Block26','Block27','Block28','Block01','Block02','Block03','Block04','Block05'"""
QUERY = f"""
WITH cte AS(
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label, 
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND RIGHT(tp.parm_Label,7) NOT IN ({EXCLUDE})
        AND family_Code = 'Q5'
    ORDER BY lot_Id, wafer_Id, parm_Label)
    
    UNION ALL
    
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X) || '_' || unitcell_Y NOT IN ('1_7','2_9','3_10','4_11','5_11','6_12','7_12','9_11','10_11','11_10','12_8','12_5','11_3','10_2','9_2','6_1','5_2','4_2','3_3','2_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND RIGHT(tp.parm_Label,7) NOT IN ({EXCLUDE})
        AND family_Code = 'Q6'
    ORDER BY lot_Id, wafer_Id, parm_Label)
),
cte2 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, retx, rety, MIN(yield) AS yield
    FROM cte
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner, retx, rety
),
cte3 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, AVG(yield) AS yield
    FROM cte2
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner
),
cte4 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, MAX(date) AS date, 
    MAX(CASE WHEN corner = '0p50V' THEN yield ELSE -100 END) AS vmin_yield,
    MAX(CASE WHEN corner = '0p65V' THEN yield ELSE -100 END) AS vnom_yield
    FROM cte3
    GROUP BY lot_id_base, lot_id, wafer_id, family_code
)

SELECT lot_id_base, lot_id, wafer_id, family_code, Max(date) AS date, vmin_yield,
    AVG(vnom_yield - vmin_yield) AS vmin_loss
FROM cte4
GROUP BY lot_id_base, lot_id, wafer_id, family_code, vmin_yield
"""
df = ibmdata.isdw.query(QUERY)
df['vmin_yield'] = df['vmin_yield'].astype(float).round(2)
df['vmin_loss'] = df['vmin_loss'].astype(float).round(2)

In [5]:
df.to_csv('~/Downloads/Sail_Vmin_exclude12345262728.csv')

All Blocks

In [7]:
DAYSBACK = 365
VMIN_CORNER = '_H2_0P50V_BLOCK'
VNOM_CORNER = '_H2_0P65V_BLOCK'
VMAX_CORNER = '_H2_1P05V_BLOCK'
QUERY = f"""
WITH cte AS(
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label, 
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND family_Code = 'Q5'
    ORDER BY lot_Id, wafer_Id, parm_Label)
    
    UNION ALL
    
    (SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X AS retx, unitcell_Y AS rety, SUBSTR(parm_Label, 10,5) AS corner,
        tw.testProgramName||RIGHT(parm_Label,8) AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) IN ('{VMIN_CORNER}','{VNOM_CORNER}'))
        AND ((unitcell_X) || '_' || unitcell_Y NOT IN ('1_7','2_9','3_10','4_11','5_11','6_12','7_12','9_11','10_11','11_10','12_8','12_5','11_3','10_2','9_2','6_1','5_2','4_2','3_3','2_4'))
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        AND family_Code = 'Q6'
    ORDER BY lot_Id, wafer_Id, parm_Label)
),
cte2 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, retx, rety, MIN(yield) AS yield
    FROM cte
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner, retx, rety
),
cte3 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, corner, MAX(date) AS date, AVG(yield) AS yield
    FROM cte2
    GROUP BY lot_id_base, lot_id, wafer_id, family_code, corner
),
cte4 AS(
    SELECT lot_id_base, lot_id, wafer_id, family_code, MAX(date) AS date, 
    MAX(CASE WHEN corner = '0p50V' THEN yield ELSE -100 END) AS vmin_yield,
    MAX(CASE WHEN corner = '0p65V' THEN yield ELSE -100 END) AS vnom_yield
    FROM cte3
    GROUP BY lot_id_base, lot_id, wafer_id, family_code
)

SELECT lot_id_base, lot_id, wafer_id, family_code, Max(date) AS date, vmin_yield,
    AVG(vnom_yield - vmin_yield) AS vmin_loss
FROM cte4
GROUP BY lot_id_base, lot_id, wafer_id, family_code, vmin_yield
"""
df = ibmdata.isdw.query(QUERY)
df['vmin_yield'] = df['vmin_yield'].astype(float).round(2)
df['vmin_loss'] = df['vmin_loss'].astype(float).round(2)

In [8]:
df.to_csv('~/Downloads/Sail_Vmin_delta.csv')